In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

# set up a new Selenium driver
from bs4 import BeautifulSoup

import os
import time
import numpy as np
import pandas as pd

In [15]:
# define the username of the profile to scrape and generate its URL
class Scraper():
    # load the URL in the Selenium driver
    def __init__(self):
        '''
        Initialize
        '''
        self.start_driver()
        try:
            os.mkdir('Timelines')
        except FileExistsError:
            print('Timelines Folder Exists')
    
    def start_driver(self):
        '''
        Start Driver
        '''
        self.driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
        self.driver.get("https://twitter.com/")
        print('Started Browser')

    def get_page(self,username):
        '''
        Get Page
        '''
        self.username = username
        URL = "https://twitter.com/" + username + "?lang=en"
        self.driver.get(URL)
        time.sleep(np.random.uniform(3,5))

    def get_html(self):
        html = self.driver.execute_script("return document.documentElement.outerHTML")
        soup = BeautifulSoup(html, 'html.parser')
        self.page=soup.prettify()
    
    def write_file(self,write_string,filename='tempfile',dir_name=False,path=''):
        if(dir_name!=False):
            print('Making Directory')
            try:
                os.mkdir(path+dir_name)
            except FileExistsError:
                print('Directory Exists')
        else:
            dir_name=''
        
        full_filename=path+dir_name+'/'+filename+'.txt'
        print('Writing to',full_filename)
        text_file = open(full_filename, "w")
        n = text_file.write(write_string)
        text_file.close()
    
    def scroll(self):
        SCROLL_PAUSE_TIME = np.random.uniform(3,8)

        # Get scroll height
        self.last_height = self.driver.execute_script("return document.body.scrollHeight")

        # Scroll down to bottom
        self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        self.new_height = self.driver.execute_script("return document.body.scrollHeight")
        
    def one_round(self):
        self.scroll()
        self.times_scrolled+=1
        self.get_html()
        self.write_file(write_string=self.page,filename='page_dump_'+str(self.times_scrolled),dir_name=self.username,path='Timelines/')
        print(self.times_scrolled,'.')

    
    def scrape_username(self,username,n_scrolls=10):
        self.get_page(username)
        self.get_html()
        self.times_scrolled=0
        self.write_file(write_string=self.page,filename='page_dump_'+str(self.times_scrolled),dir_name=self.username,path='Timelines/')
        self.driver.save_screenshot('Timelines/'+self.username+'/'+self.username+'.png')

        self.not_scrolling=0
        while(self.times_scrolled<n_scrolls):
            self.one_round()
            
            if(self.last_height == self.new_height):#Stop Scrolling if its not moving forward
                self.not_scrolling+=1
                if(self.not_scrolling>1):
                    break
    
        
        
        




In [14]:
S=Scraper()

Started Browser
Timelines Folder Exists


In [1]:
#Now enter your username and password

In [ ]:
handle=''
S.scrape_username(handle)